In [23]:
import os
import pandas as pd
import numpy as np
import uncertainties as unc

\begin{equation}
    Q = 2k\pi l \frac{\Delta T}{\ln{r_2} - \ln{r_1}}\\
\end{equation}

We are given:<br> 
$0.17 \leq k \leq 0.20, k = 0.185 \pm 0.015 \frac{W}{m K}$<br>
$l = 11.25 \times .0254 \pm 0.001 \times .0254 \text{m}$ <br>
$\Delta T = 40/60 - (24.1 \pm\ 0.05) = 15.9/35.9 \pm 0.05 ^{\circ}\text{C}$<br>
Outer diameter: $8 \times .0254 \pm 0.045 \times .0254 \text{m}$ <br>
Wall Thickness: $0.375 \times .0254 \pm 0.05625 \times .0254 \text{m}$ (15% of 0.375)<br>



Each value with its uncertainty is initialized below

In [62]:
k, k_u = 0.185, 0.015
l, l_u = 0.28575, 2.54*10**(-5) #11.25 * 0.0254, 0.001 * .0254
# Outer diameter is 8" +- 0.045"
# Wall thickness is 3/8" +- 0.05625" (15%)
# Divide diameter by 2 to get outer radius,
# Outer radius is 4" +- 0.0225
r2, r2_u = 4 * .0254, 0.0225 * .0254
# Subtract thickness from outer radius to get inner radius, add uncertainties
# Inner radius is 3.625" +- (0.0225 + 0.05625)
r1, r1_u = 3.625 * .0254, (0.0225 + 0.05625) * .0254

We use the uncertainties package to create objects that perform error propagation for us during operations (I still verified that the uncertainty calculations were correct afterwards). Note that due to the self-reference in the declarations, you will receive an error if you run this cell multiple times without running the previous code cell above.

In [63]:
from uncertainties import ufloat
import math

k = ufloat(k, k_u)
l = ufloat(l, l_u)
dT = ufloat(15.9, 0.1) # Internal Temperature = 40 degrees
r1 = ufloat(r1, r1_u)
r2 = ufloat(r2, r2_u)

Plugging all of our values:

In [64]:
from uncertainties import umath
Q = 2 * k * math.pi * l * dT / umath.log(r2 / r1)
print(Q.n) # Actual value
print(Q.s) # Uncertainty/standard deviation

53.649173267163725
12.984897557604475


T = 40:<br>
53.649173267163725<br>
12.984897557604475<br>

T = 60:<br>
121.13241008120613<br>
29.310144280180875<br>

For 40 degrees, $Q = 50 \pm 10 W$<br>
For 60 degrees, $Q = 120 \pm 30 W$<br>
This is for the heat loss through the walls. These values are expectedly less than the values we obtained
experimentally, which would account for heat loss through the top and bottom plates as well.

These are the actual heat losses obtained from the experiment, see notes.txt for a further explanation. We use the data from B(old) because it fits better with our expected values (heat loss increases with pressure).

![heat_loss_summary.jpg](heat_loss_summary.png)

Each time value has an uncertainty of $0.05s$ and each energy reading has an uncertainty of $0.05 kJ$. Therefore we need to make uncertainty objects reflecting these considerations.

In [71]:
# 40C, 40PSI
ta, Qa = ufloat(300.5, 0.1), ufloat(22.7 * 1000, 0.1 * 1000)
# 40C, 70PSI
tb, Qb = ufloat(145.4, 0.1), ufloat(12.2 * 1000, 0.1 * 1000)
# 60C, 40PSI
tc, Qc = ufloat(205.5, 0.1), ufloat(33.0 * 1000, 0.1 * 1000)
# 60C, 70PSI
td, Qd = ufloat(222.0, 0.1), ufloat(48.2 * 1000, 0.1 * 1000)

Wa = Qa/ta
Wb = Qb/tb
Wc = Qc/tc
Wd = Qd/td

print(f"Trial A: {Wa.n} +/- {Wa.s}")
print(f"Trial B: {Wb.n} +/- {Wb.s}")
print(f"Trial C: {Wc.n} +/- {Wc.s}")
print(f"Trial D: {Wd.n} +/- {Wd.s}")


Trial A: 75.54076539101497 +/- 0.333726836873501
Trial B: 83.90646492434662 +/- 0.6901746722307014
Trial C: 160.5839416058394 +/- 0.49285232771585674
Trial D: 217.11711711711712 +/- 0.46094527520991324


Trial A: 75.54076539101497 +/- 0.333726836873501<br>
Trial B: 83.90646492434662 +/- 0.6901746722307014<br>
Trial C: 160.5839416058394 +/- 0.49285232771585674<br>
Trial D: 217.11711711711712 +/- 0.46094527520991324<br>

Trial A: $75.5 \pm 0.3$<br>
Trial B: $83.9 \pm 0.7$<br>
Trial C: $160.6 \pm 0.5$<br>
Trial D: $217.1 \pm 0.5$<br>

Subtracting the calculated heat loss through the cylinder walls, we get the following heat losses through the top and bottom of the cylinder:<br>
Trial A: $21.9 \pm 13.3$<br>
Trial B: $30.3 \pm 13.7$<br>
Trial C: $39.5 \pm 29.8$<br>
Trial D: $

Extra: Verify calculations for Uncertainty Propagation:

![uncertainty propagation](uncertainty_propagation.jpg)

In [69]:
dQdk, dk = 2 * math.pi * 15.9 * l.n/ math.log(r2.n/r1.n), k.s
dQdT, dT = 2 * math.pi * k.n * l.n / math.log(r2.n/r1.n), 0.1
dQdl, dl = 2 * math.pi * k.n * 15.9 / math.log(r2.n/r1.n), l.s
dQdr1, dr1 = 2 * math.pi * k.n * l.n * 15.9 / (r1.n*(math.log(r2.n/r1.n)**2)), r1.s
dQdr2, dr2 = -2 * math.pi * k.n * l.n * 15.9 / (r2.n*(math.log(r2.n/r1.n)**2)), r2.s

u = math.sqrt((dQdk * dk)**2 + (dQdT * dT)**2 + (dQdl * dl)**2 + (dQdr1 * dr1)**2 + (dQdr2 * dr2)**2)
print(u) # returns 12.9849, equal to the uncertainty we got using our uncertainties package.


12.984897557604473
